# London Property Pricing Research

This is the descriptive model for determining the property price influence factors in London. It analyzes the city’s regions for their property price, their geographical location and the venues nearby. The model clusters the city by the property price to give the insight on the prices' spreading over the territory, acquires the venues list for each region and finds the correlation between different venues’ presence and its impact on the property pricing.

There data sources for the model are: the London Datastore website for the property pricing over the city regions, the ArcGIS Hub for the London regions’ geodata, and the Forsquare API for venues’ data.

In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Acquiring the wards reference data

In [3]:
df_wards_reference_raw=pd.read_csv('https://opendata.arcgis.com/datasets/7193daa99995445aa84a0b23352e56a1_0.csv?outSR=%7B%22latestWkid%22%3A4326%2C%22wkid%22%3A4326%7D')

In [4]:
df_wards_reference_raw.head()

,objectid,wd17cd,wd17nm,wd17nmw,bng_e,bng_n,long,lat,st_areashape,st_lengthshape
0,1,E05000026,Abbey,,544434,184378,0.081291,51.539822,0.000163,0.095056
1,2,E05000027,Alibon,,549247,185196,0.150987,51.545921,0.000177,0.081044
2,3,E05000028,Becontree,,546863,185869,0.116912,51.552601,0.000167,0.075036
3,4,E05000029,Chadwell Heath,,548266,189340,0.138596,51.583420,0.000439,0.108753
4,5,E05000030,Eastbrook,,550774,186272,0.173453,51.555191,0.000447,0.106966


## Acquiring the property prices

In [5]:
df_prices = pd.read_excel('https://data.london.gov.uk/download/average-house-prices/fb8116f5-06f8-42e0-aa6c-b0b1bd69cdba/land-registry-house-prices-ward.xls', sheet_name='Mean')

In [6]:
df_prices.head()

,New code,Ward name,Borough name,Year ending Dec 1995,Year ending Mar 1996,Year ending Jun 1996,Year ending Sep 1996,Year ending Dec 1996,Year ending Mar 1997,Year ending Jun 1997,...,Year ending Sep 2015,Year ending Dec 2015,Year ending Mar 2016,Year ending Jun 2016,Year ending Sep 2016,Year ending Dec 2016,Year ending Mar 2017,Year ending Jun 2017,Year ending Sep 2017,Year ending Dec 2017
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E09000001,City of London,City of London,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,E05000026,Abbey,Barking and Dagenham,51077.6,49868.8,49901.6,51935.1,50766.5,50189.5,47807.5,...,200307,205772,213651,221505,230617,255508,257490,260566,265572,259487
3,E05000027,Alibon,Barking and Dagenham,45490.4,44701.5,44486,45894.1,46145.1,47019.2,48608.8,...,220636,224947,244132,258467,275653,288472,287671,282210,284011,283397
4,E05000028,Becontree,Barking and Dagenham,48947.6,49655.6,50129.2,51113.3,52333.8,54958.4,56108.9,...,222996,245902,253269,269184,277961,282562,295482,293541,290885,291677


## Creating the initial dataset (wards, prices, coordinates)

In [287]:
# ------- Filtering out the wards' borders' coordinates -------

df_wards_reference = df_wards_reference_raw[['wd17nm', 'long', 'lat']].rename(columns = {'wd17nm': 'Name', 'long': 'Longitude', 'lat': 'Latitude'})

# ------- Filtering out the wards' names and the most recent pricing data -------

df_prices_filtered = df_prices.drop(df_prices.index[0])
df_prices_filtered = df_prices_filtered[['Ward name', 'Year ending Dec 2017']]
df_prices_filtered.rename(columns = {'Ward name': 'Name', 'Year ending Dec 2017': 'Price'}, inplace = True)
df_prices_filtered = df_prices_filtered.drop(df_prices.index[1])
df_prices_filtered.drop_duplicates(subset=['Name'], keep=False, inplace = True)
df_prices_filtered.reset_index(drop = True, inplace = True)

# ------- Getting the latitudes -------

# Getting the wards' borders' latitudes
df_codes_n_prices = pd.merge(df_prices_filtered[['Name']], df_wards_reference[['Name', 'Latitude']], on = 'Name')

# Getting the wards' average latitudes
df_codes_n_prices_grouped = df_codes_n_prices.groupby(['Name'])['Latitude'].apply(list)
df_codes_n_prices_grouped = df_codes_n_prices_grouped.reset_index()
df_codes_n_prices_grouped['Latitude'] = pd.DataFrame(df_codes_n_prices_grouped['Latitude'].values.tolist()).mean(1)

# ------- Getting the longitudes -------

# Getting the wards' borders' longitudes
df_codes_n_prices_lon = pd.merge(df_prices_filtered[['Name']], df_wards_reference[['Name', 'Longitude']], on = 'Name')

# Getting the wards' average latitudes
df_codes_n_prices_grouped_lon = df_codes_n_prices_lon.groupby(['Name'])['Longitude'].apply(list)
df_codes_n_prices_grouped_lon = df_codes_n_prices_grouped_lon.reset_index()
df_codes_n_prices_grouped_lon['Longitude'] = pd.DataFrame(df_codes_n_prices_grouped_lon['Longitude'].values.tolist()).mean(1)

# ------- Forming the resulting dataset -------
df = pd.merge(df_prices_filtered, df_codes_n_prices_grouped, on = 'Name')
df = pd.merge(df, df_codes_n_prices_grouped_lon, on = 'Name')
df['Price'] = df['Price'].astype(int)
df.head()

,Name,Price,Latitude,Longitude
0,Alibon,283396,51.545921,0.150987
1,Becontree,291677,51.552601,0.116912
2,Chadwell Heath,302849,51.583420,0.138596
3,Eastbrook,326229,51.194561,-0.026804
4,Eastbury,317988,51.535809,0.104756


## Visualizing the wards' locations

In [200]:
address = 'London, GB'

geolocator = Nominatim(user_agent="london_expl")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, name, price in zip(df['Latitude'], df['Longitude'], df['Name'], df['Price']):
    label = 'Ward: {}, average property price: {}'.format(name, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)
    
map_london

## Clustering

In [288]:
kclusters = 10
df_for_clustering =  df.drop('Name', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_for_clustering)

kmeans.labels_[0:10]

array([4, 4, 4, 4, 4, 4, 4, 4, 8, 4], dtype=int32)

In [289]:
df.insert(0, 'Cluster', kmeans.labels_)

In [290]:
df.head()

,Cluster,Name,Price,Latitude,Longitude
0,4,Alibon,283396,51.545921,0.150987
1,4,Becontree,291677,51.552601,0.116912
2,4,Chadwell Heath,302849,51.583420,0.138596
3,4,Eastbrook,326229,51.194561,-0.026804
4,4,Eastbury,317988,51.535809,0.104756


## Visualizing the clusters

In [331]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lng, name, price, cluster in zip(df['Latitude'], df['Longitude'], df['Name'], df['Price'], df['Cluster']):
    label = folium.Popup(str(name) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Grouping the clusters by price

In [291]:
df_clusters = df.drop(['Name', 'Latitude', 'Longitude'], 1)
df_clusters = df_clusters.groupby(['Cluster'])['Price'].mean()
df_clusters = df_clusters.reset_index()
df_clusters_sorted = df_clusters.sort_values(by=['Price'], ascending=False)

In [292]:
df_clusters_sorted

,Cluster,Price
2,2,4.079424e+06
9,9,2.891610e+06
5,5,2.091560e+06
1,1,1.547812e+06
7,7,1.176190e+06
3,3,9.299517e+05
6,6,7.218964e+05
0,0,5.787742e+05
8,8,4.617621e+05
4,4,3.467885e+05


## Getting the venues for the wards

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, end='\r')
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
# @hidden_cell

CLIENT_ID = 'EA0M5Z54HBMWTG0MKLUZKEGMEF2FMBXU3GPAXMWZOLF053HN'
CLIENT_SECRET = 'BKJUV0JWEISUTH5D1NRDA5D50GFBQIVEOLVH10U0RNC5XOQG'
VERSION = '20180604'

In [17]:
LIMIT = 100
radius = 500

london_venues = getNearbyVenues(names=df['Name'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 398 uniques categories.


In [293]:
london_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alibon,51.545921,0.150987,Lidl,51.542819,0.148625,Supermarket
1,Alibon,51.545921,0.150987,Iceland,51.542006,0.147933,Grocery Store
2,Alibon,51.545921,0.150987,The Lord Denman,51.542375,0.148090,Pub
3,Alibon,51.545921,0.150987,Londis,51.547309,0.145934,Convenience Store
4,Alibon,51.545921,0.150987,Papa John's Pizza,51.542671,0.147628,Pizza Place


In [294]:
df_venues_clustered = pd.merge(london_venues, df[['Name', 'Cluster', 'Price']], left_on = 'Borough', right_on = 'Name')
df_venues_clustered =  df_venues_clustered.drop('Name', 1)
df_venues_clustered

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster,Price
0,Alibon,51.545921,0.150987,Lidl,51.542819,0.148625,Supermarket,4,283396
1,Alibon,51.545921,0.150987,Iceland,51.542006,0.147933,Grocery Store,4,283396
2,Alibon,51.545921,0.150987,The Lord Denman,51.542375,0.148090,Pub,4,283396
3,Alibon,51.545921,0.150987,Londis,51.547309,0.145934,Convenience Store,4,283396
4,Alibon,51.545921,0.150987,Papa John's Pizza,51.542671,0.147628,Pizza Place,4,283396
...,...,...,...,...,...,...,...,...,...
8065,Warwick,51.492279,-0.143920,Ebury Square Gardens,51.491948,-0.150040,Garden,3,987345
8066,Warwick,51.492279,-0.143920,Caffè Nero,51.492191,-0.140128,Coffee Shop,3,987345
8067,Warwick,51.492279,-0.143920,Belgrave Hotel,51.490256,-0.138052,Hotel,3,987345
8068,Warwick,51.492279,-0.143920,Gastronomica,51.491963,-0.138379,Italian Restaurant,3,987345


## Counting the average occurance frequencies for each venue

In [295]:
dummies = pd.get_dummies(df_venues_clustered[['Venue Category']], prefix="", prefix_sep="")
dummies['Cluster'] = df_venues_clustered['Cluster']
london_grouped = dummies.groupby('Cluster').mean().reset_index()
london_grouped_transposed = london_grouped.T.drop('Cluster', 0)
london_grouped_transposed['Average Frequencies'] = london_grouped_transposed.mean(axis=1)
venues_freq = london_grouped_transposed[['Average Frequencies']]
venues_freq.head()

,Average Frequencies
ATM,0.000069
Accessories Store,0.000310
Afghan Restaurant,0.000186
African Restaurant,0.000471
Airport,0.000162


## Getting the top 50 venues

In [296]:
top_venues = venues_freq.sort_values(by=['Average Frequencies'], ascending=False).head(50)
top_venues.reset_index(inplace = True)
top_venues.rename(columns = {'index': 'Categories'}, inplace = True)
top_venues.head()

,Categories,Average Frequencies
0,Pub,0.070385
1,Café,0.054670
2,Coffee Shop,0.051463
3,Hotel,0.041896
4,Grocery Store,0.032031


## Getting the frequencies by clusters dataset

In [297]:
# Selecting the features
features_lst = ['Cluster'] + top_venues['Categories'].tolist()
top_venues_categories = london_grouped[features_lst]
top_venues_categories

,Cluster,Pub,Café,Coffee Shop,Hotel,Grocery Store,Italian Restaurant,Park,Indian Restaurant,Pizza Place,...,Asian Restaurant,Sushi Restaurant,Mediterranean Restaurant,Furniture / Home Store,Greek Restaurant,Yoga Studio,Wine Bar,Department Store,Plaza,Farmers Market
0,0,0.078821,0.062371,0.065113,0.028787,0.044551,0.019877,0.034270,0.023989,0.023989,...,0.001371,0.002056,0.006169,0.002742,0.002742,0.004798,0.000685,0.001371,0.004112,0.002742
1,1,0.079940,0.052790,0.049774,0.067873,0.024133,0.042232,0.010558,0.016591,0.019608,...,0.004525,0.001508,0.003017,0.001508,0.012066,0.009050,0.006033,0.000000,0.000000,0.003017
2,2,0.045455,0.053030,0.030303,0.037879,0.007576,0.045455,0.000000,0.007576,0.000000,...,0.007576,0.007576,0.000000,0.000000,0.015152,0.007576,0.007576,0.022727,0.000000,0.007576
3,3,0.083803,0.065270,0.071716,0.043513,0.029009,0.035455,0.024980,0.010475,0.029009,...,0.004835,0.007252,0.003223,0.005641,0.001612,0.005641,0.004029,0.000806,0.007252,0.002417
4,4,0.065058,0.035677,0.037775,0.025184,0.062959,0.008395,0.043022,0.051417,0.014690,...,0.008395,0.000000,0.004197,0.013641,0.000000,0.000000,0.000000,0.006296,0.003148,0.001049
5,5,0.056701,0.041237,0.041237,0.082474,0.015464,0.048969,0.007732,0.018041,0.025773,...,0.010309,0.010309,0.010309,0.002577,0.000000,0.007732,0.000000,0.007732,0.015464,0.005155
6,6,0.094090,0.075428,0.060653,0.026439,0.047434,0.026439,0.032659,0.023328,0.027216,...,0.004666,0.003110,0.004666,0.003888,0.000778,0.004666,0.003888,0.001555,0.001555,0.005443
7,7,0.087171,0.075658,0.077303,0.032895,0.026316,0.026316,0.027961,0.019737,0.013158,...,0.003289,0.004934,0.004934,0.004934,0.003289,0.004934,0.001645,0.003289,0.011513,0.006579
8,8,0.075426,0.047851,0.052717,0.017843,0.053528,0.020276,0.049473,0.027575,0.025953,...,0.003244,0.001622,0.004055,0.004055,0.002433,0.002433,0.004055,0.002433,0.002433,0.000811
9,9,0.037383,0.037383,0.028037,0.056075,0.009346,0.037383,0.018692,0.028037,0.028037,...,0.009346,0.018692,0.009346,0.009346,0.009346,0.000000,0.018692,0.000000,0.000000,0.009346


## Getting the correlation

In [326]:
correlations_lst = []
for feature in top_venues_categories.columns[1:]:
    df_temp = pd.merge(df_clusters_sorted[['Cluster', 'Price']], top_venues_categories[['Cluster', feature]], on = 'Cluster')
    df_temp.drop(columns = ['Cluster'], inplace = True)
    correlations_lst.append(df_temp.corr(method='pearson').iloc[[0]][feature].values[0])

df_venues_corr = pd.DataFrame({'Venue': features, 'Correlation': correlations_lst})
df_venues_corr

,Venue,Correlation
0,Pub,-0.766895
1,Café,-0.271336
2,Coffee Shop,-0.633679
3,Hotel,0.462592
4,Grocery Store,-0.874103
5,Italian Restaurant,0.759050
6,Park,-0.845409
7,Indian Restaurant,-0.502731
8,Pizza Place,-0.492642
9,Bakery,0.301393


## Finding the features with highest positive correlation

In [319]:
df_pos = df_venues_corr.loc[df_venues_corr['Correlation'] > 0].sort_values(by=['Correlation'], ascending=False).head(10)
df_pos.reset_index(drop = True, inplace = True)
df_pos

,Venue,Correlation
0,Juice Bar,0.964597
1,Garden,0.877125
2,Restaurant,0.875747
3,Art Gallery,0.814109
4,Farmers Market,0.774828
5,Greek Restaurant,0.774135
6,Italian Restaurant,0.759050
7,Chinese Restaurant,0.739535
8,French Restaurant,0.705209
9,Japanese Restaurant,0.702539


## Finding the features with highest negative correlation

In [320]:
df_neg = df_venues_corr.loc[df_venues_corr['Correlation'] < 0].sort_values(by=['Correlation'], ascending=True).head(10)
df_neg.reset_index(drop = True, inplace = True)
df_neg

,Venue,Correlation
0,Grocery Store,-0.874103
1,Park,-0.845409
2,Convenience Store,-0.771125
3,Pub,-0.766895
4,Train Station,-0.712171
5,Fast Food Restaurant,-0.695274
6,Coffee Shop,-0.633679
7,Supermarket,-0.630858
8,Bus Stop,-0.575454
9,Bar,-0.523705


## Outcome

The property costs higher, if any of these venues are within 500 m from it:
    - Juice Bar
    - Garden
    - Restaurant
    - Art Gallery
    - Farmers Market
    - Greek Restaurant
    - Italian Restaurant
    - Chinese Restaurant
    - French Restaurant
    - Japanese Restaurant
    
Also, the following property venue types influence the property proce negatively:
    - Grocery Store
    - Park
    - Convenience Store
    - Pub
    - Train Station
    - Fast Food Restaurant
    - Coffee Shop
    - Supermarket
    - Bus Stop
    - Bar
    
Besides that, the influence of the venues' geografical location can be visually estimated by examining the clusters spread map given above